In [1]:
import pdal
import glob
import geopandas as gpd
import numpy as np

In [2]:
files = glob.glob('downloads/LiDAR/*.laz')

In [3]:
scms = [file[20:][:8] for file in files]

In [4]:
gdf_articulacao = gpd.read_file("zip://downloads/SIRGAS_SHP_quadriculamdt.zip!/SIRGAS_SHP_quadriculamdt/")

In [5]:
_ = gdf_articulacao.set_crs(epsg=31983, inplace=True)

In [10]:
resolution = 0.5

def dem_pipeline(scm, width, height, origin_x, origin_y, resolution):
    return [
        {
            "type": "readers.las",
            "filename": f'downloads/LiDAR/MDS_{scm}_1000.laz',
            "override_srs": "EPSG:31983"
        },
        {
            "type":"filters.range",
            "limits":"Classification[2:2]"
        },
        {
            "type": "filters.delaunay"
        },
        {
            "type": "filters.faceraster",
            "resolution":resolution,
            "width": width,
            "height": height,
            "origin_x": origin_x,
            "origin_y": origin_y,
        },        
        {
            "filename":f"results/DEM/MDT-{scm}-50cm.tiff",
            "gdaldriver":"GTiff",
            "type": "writers.raster",
            "gdalopts":"COMPRESS=ZSTD, PREDICTOR=3, BIGTIFF=YES",
            "nodata":"0",
            "data_type": "float32",
            # "default_srs": "EPSG:31983"
        }
    ]

def laz_pipeline(scm, width, height, origin_x, origin_y, resolution):
    return [
        {
            "type":"readers.las",
            "filename":f"downloads/LiDAR/MDS_{scm}_1000.laz"
        },
        {
            "filename":f"sresults/BHM-Z-{scm}.tiff",
            "gdaldriver":"GTiff",
            "width": width,
            "height": height,
            "origin_x": origin_x,
            "origin_y": origin_y,
            "radius": f'{resolution * 2 * np.sqrt(2)}',
            "override_srs": "EPSG:31983",
            "output_type":"max",
            "resolution":resolution,
            "dimension": "Z",
            "data_type": "float32",
            "type": "writers.gdal",
            "gdalopts":"COMPRESS=ZSTD, PREDICTOR=3, BIGTIFF=YES",
            "where": "(Classification == 6)",
        },
        {
            "type":"filters.hag_dem",
            "raster": "results/DEM/MDT-{scm}-50cm.tiff"
        },
        {
            "filename":f"results/BHM-{scm}.tiff",
            "gdaldriver":"GTiff",
            "output_type":"max",
            "resolution": resolution,
            "radius": f'{resolution * 2 * np.sqrt(2)}',
            "dimension":"HeightAboveGround",
            "width": width,
            "height": height,
            "origin_x": origin_x,
            "origin_y": origin_y,
            # "nodata":"0",
            "data_type": "float32",
            "type": "writers.gdal",
            "where": "(Classification == 6)",
            "override_srs": "EPSG:31983"
        },
        {
            "filename":f"results/VHM-{scm}.tiff",
            "gdaldriver":"GTiff",
            "output_type":"max",
            "resolution":resolution,
            "radius": f'{resolution * 2 * np.sqrt(2)}',
            "dimension":"HeightAboveGround",
            "width": 1077,
            "height": 1166,
            "origin_x": 323586,
            "origin_y": 7386800,
            # "nodata":"0",
            "data_type": "float32",
            "type": "writers.gdal",
            "where": "(Classification == 4) || (Classification == 5)",
            "override_srs": "EPSG:31983"
        },
        {
            "type":"filters.range",
            "limits":"Classification[6:6]"
        },
        {
            "type":"filters.voxeldownsize",
            "cell":0.5,
            "mode":"center"
        },
        {
            "type":"filters.dbscan",
            "min_points":5,
            "eps": (resolution + 0.10) * np.sqrt(2),
            "dimensions":"X,Y,Z"
        },
        {
            "type":"filters.ferry",
            "dimensions":"HeightAboveGround => Z"
        },
        {
            "type":"writers.las",
            "filename":f"sresults/Cluster-{scm}.laz",
            "extra_dims": "all",
            # "output_dims":"X,Y,Z,ClusterID"
        },
    ]

In [11]:
for _, scm in gdf_articulacao.loc[gdf_articulacao.qmdt_cod.isin(scms)].iterrows():
    print(scm.qmdt_cod)
    coords = [[xy[0], xy[1]] for xy in scm.geometry.exterior.coords]
    xy_max = np.max(np.array(coords), axis=0) 
    xy_min = np.min(np.array(coords), axis=0)
    width_height = np.ceil(xy_max * 2) - np.ceil(xy_min * 2)
    # print(width_height)
    origin_xy = np.floor(xy_min * 2)/2
    # print(origin_xy)
    dem = dem_pipeline(scm.qmdt_cod, width_height[0], width_height[1], xy_min[0], xy_min[1], resolution)

    pipeline = pdal.Pipeline(json.dumps(laz))
    # pipeline.validate()   
    n_points = pipeline.execute()
    print(f'Pipeline selected {n_points} points')
    
    laz = laz_pipeline(scm.qmdt_cod, width_height[0], width_height[1], xy_min[0], xy_min[1], resolution)

    break



3313-144


In [12]:
laz

[{'type': 'readers.las', 'filename': 'downloads/LiDAR/MDS_3313-144_1000.laz'},
 {'filename': 'sresults/BHM-Z-3313-144.tiff',
  'gdaldriver': 'GTiff',
  'width': 1077.0,
  'height': 1167.0,
  'origin_x': 321914.703,
  'origin_y': 7393125.297,
  'radius': '1.4142135623730951',
  'override_srs': 'EPSG:31983',
  'output_type': 'max',
  'resolution': 0.5,
  'dimension': 'Z',
  'data_type': 'float32',
  'type': 'writers.gdal',
  'gdalopts': 'COMPRESS=ZSTD, PREDICTOR=3, BIGTIFF=YES',
  'where': '(Classification == 6)'},
 {'type': 'filters.hag_dem', 'raster': 'results/DEM/MDT-{scm}-50cm.tiff'},
 {'filename': 'results/BHM-3313-144.tiff',
  'gdaldriver': 'GTiff',
  'output_type': 'max',
  'resolution': 0.5,
  'radius': '1.4142135623730951',
  'dimension': 'HeightAboveGround',
  'width': 1077.0,
  'height': 1167.0,
  'origin_x': 321914.703,
  'origin_y': 7393125.297,
  'data_type': 'float32',
  'type': 'writers.gdal',
  'where': '(Classification == 6)',
  'override_srs': 'EPSG:31983'},
 {'filenam

In [71]:
xy_max

array([ 321928.75 , 7393125.297])

In [72]:
xy_min

array([ 321390.036, 7392542.026])

In [73]:
xy_max - xy_min

array([538.714, 583.271])